In [ ]:
# import os
# import pandas as pd
# from PIL import Image
# import io

# input_parquet = '/home/seif_elkerdany/projects/data/cleaned_data38K.parquet'
# df = pd.read_parquet(input_parquet)

# base_output_dir = "/home/seif_elkerdany/projects/data/38K_images"
# img1_dir = os.path.join(base_output_dir, "image1")
# img2_dir = os.path.join(base_output_dir, "image2")

# os.makedirs(img1_dir, exist_ok=True)
# os.makedirs(img2_dir, exist_ok=True)

# def extract_image_bytes(image_cell, col_name, index):
#     """
#     Extract image bytes from a cell.
#     If the cell is a dict and contains the key 'bytes', return its value.
#     Otherwise, assume the cell is already the raw bytes.
#     """
#     if isinstance(image_cell, dict):
#         if 'bytes' in image_cell:
#             return image_cell['bytes']
#         else:
#             if len(image_cell) == 1:
#                 return next(iter(image_cell.values()))
#             else:
#                 print(f"{col_name} at index {index} is a dict with keys {list(image_cell.keys())}.")
#                 return None
#     return image_cell


# for index, row in df.iterrows():
    
#     raw_image1 = row['image1']
#     raw_image2 = row['image2']

#     image1_bytes = extract_image_bytes(raw_image1, 'image1', index)
#     image2_bytes = extract_image_bytes(raw_image2, 'image2', index)

   
#     if image1_bytes is None or image2_bytes is None:
#         continue

    
#     try:
#         image1 = Image.open(io.BytesIO(image1_bytes))
#     except Exception as e:
#         print(f"Error processing image1 at index {index}: {e}")
#         continue

#     try:
#         image2 = Image.open(io.BytesIO(image2_bytes))
#     except Exception as e:
#         print(f"Error processing image2 at index {index}: {e}")
#         continue

    
#     image1_filename = f"img1_{index}.png"  
#     image2_filename = f"img2_{index}.png"
#     image1_path = os.path.join(img1_dir, image1_filename)
#     image2_path = os.path.join(img2_dir, image2_filename)

#     try:
#         image1.save(image1_path)
#     except Exception as e:
#         print(f"Error saving image1 at index {index}: {e}")
#         continue

#     try:
#         image2.save(image2_path)
#     except Exception as e:
#         print(f"Error saving image2 at index {index}: {e}")
#         continue

#     df.at[index, 'image1'] = image1_path
#     df.at[index, 'image2'] = image2_path

# output_csv = "/home/seif_elkerdany/projects/data/38K_dataset.csv"
# df.to_csv(output_csv, index=False)

# print("Dataset updated successfully! The images are stored in the folder:", base_output_dir)

Dataset updated successfully! The images are stored in the folder: /home/seif_elkerdany/projects/data/38K_images


In [ ]:
# df.head()

,image1,image2,target
0,/home/seif_elkerdany/projects/data/38K_images/...,/home/seif_elkerdany/projects/data/38K_images/...,1
1,/home/seif_elkerdany/projects/data/38K_images/...,/home/seif_elkerdany/projects/data/38K_images/...,1
2,/home/seif_elkerdany/projects/data/38K_images/...,/home/seif_elkerdany/projects/data/38K_images/...,1
3,/home/seif_elkerdany/projects/data/38K_images/...,/home/seif_elkerdany/projects/data/38K_images/...,1
4,/home/seif_elkerdany/projects/data/38K_images/...,/home/seif_elkerdany/projects/data/38K_images/...,1


In [1]:
import pandas as pd

In [2]:
df_38K = pd.read_csv("/home/seif_elkerdany/projects/data/38K_dataset.csv")
df_hijab = pd.read_csv("/home/seif_elkerdany/projects/Dataset/siamese_dataset.csv")
df_student = pd.read_csv("/home/seif_elkerdany/projects/data/student/students_data.csv")
df_lfw = pd.read_csv("/home/seif_elkerdany/projects/data/lfw/pairs.csv")

In [3]:
df_38K.head()

,image1,image2,target
0,/home/seif_elkerdany/projects/data/38K_images/...,/home/seif_elkerdany/projects/data/38K_images/...,1
1,/home/seif_elkerdany/projects/data/38K_images/...,/home/seif_elkerdany/projects/data/38K_images/...,1
2,/home/seif_elkerdany/projects/data/38K_images/...,/home/seif_elkerdany/projects/data/38K_images/...,1
3,/home/seif_elkerdany/projects/data/38K_images/...,/home/seif_elkerdany/projects/data/38K_images/...,1
4,/home/seif_elkerdany/projects/data/38K_images/...,/home/seif_elkerdany/projects/data/38K_images/...,1


In [4]:
df_hijab.head()

,image1,image2,label
0,/home/seif_elkerdany/projects/Dataset/Hijab/85...,/home/seif_elkerdany/projects/Dataset/Hijab/au...,1
1,/home/seif_elkerdany/projects/Dataset/Hijab/31...,/home/seif_elkerdany/projects/Dataset/Hijab/au...,1
2,/home/seif_elkerdany/projects/Dataset/Hijab/61...,/home/seif_elkerdany/projects/Dataset/Hijab/au...,1
3,/home/seif_elkerdany/projects/Dataset/Hijab/it...,/home/seif_elkerdany/projects/Dataset/Hijab/au...,1
4,/home/seif_elkerdany/projects/Dataset/Hijab/is...,/home/seif_elkerdany/projects/Dataset/Hijab/au...,1


In [5]:
df_student.head()

,path1,path2,label
0,23101028_1.jpg,23101028_2.jpg,1
1,23101028_1.jpg,23101028_3.jpg,1
2,23101028_1.jpg,23101455_1.jpg,0
3,23101028_1.jpg,23101473_2.jpg,0
4,23101028_1.jpg,23101455_4.jpg,0


In [6]:
df_lfw.head()

,image_1,image_2,label
0,/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw...,/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw...,1
1,/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw...,/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw...,1
2,/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw...,/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw...,1
3,/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw...,/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw...,1
4,/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw...,/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw...,1


In [7]:
old_path = '/kaggle/input/lfw-dataset/lfw-deepfunneled/'
new_path = '/home/seif_elkerdany/projects/data/lfw/'

df_lfw['image_1'] = df_lfw['image_1'].str.replace(old_path, new_path)
df_lfw['image_2'] = df_lfw['image_2'].str.replace(old_path, new_path)

df_lfw.rename(columns={'image_1': 'image1', 'image_2':'image2'}, inplace=True)

df_lfw.head()

,image1,image2,label
0,/home/seif_elkerdany/projects/data/lfw/lfw-dee...,/home/seif_elkerdany/projects/data/lfw/lfw-dee...,1
1,/home/seif_elkerdany/projects/data/lfw/lfw-dee...,/home/seif_elkerdany/projects/data/lfw/lfw-dee...,1
2,/home/seif_elkerdany/projects/data/lfw/lfw-dee...,/home/seif_elkerdany/projects/data/lfw/lfw-dee...,1
3,/home/seif_elkerdany/projects/data/lfw/lfw-dee...,/home/seif_elkerdany/projects/data/lfw/lfw-dee...,1
4,/home/seif_elkerdany/projects/data/lfw/lfw-dee...,/home/seif_elkerdany/projects/data/lfw/lfw-dee...,1


In [8]:
df_student.rename(columns={'path1': 'image1', 'path2':'image2'}, inplace=True)

df_student.head()

,image1,image2,label
0,23101028_1.jpg,23101028_2.jpg,1
1,23101028_1.jpg,23101028_3.jpg,1
2,23101028_1.jpg,23101455_1.jpg,0
3,23101028_1.jpg,23101473_2.jpg,0
4,23101028_1.jpg,23101455_4.jpg,0


In [9]:
new_path = "/home/seif_elkerdany/projects/data/student/DataSet_Grey/"

df_student['image1'] = new_path + df_student['image1']
df_student['image2'] = new_path + df_student['image2']

df_student.head()

,image1,image2,label
0,/home/seif_elkerdany/projects/data/student/Dat...,/home/seif_elkerdany/projects/data/student/Dat...,1
1,/home/seif_elkerdany/projects/data/student/Dat...,/home/seif_elkerdany/projects/data/student/Dat...,1
2,/home/seif_elkerdany/projects/data/student/Dat...,/home/seif_elkerdany/projects/data/student/Dat...,0
3,/home/seif_elkerdany/projects/data/student/Dat...,/home/seif_elkerdany/projects/data/student/Dat...,0
4,/home/seif_elkerdany/projects/data/student/Dat...,/home/seif_elkerdany/projects/data/student/Dat...,0


In [10]:
df_38K.rename(columns={'target': 'label'}, inplace=True)

In [ ]:
# # Sample fractions from each DataFrame
# sample_fraction_38K   = df_38K.sample(frac=0.1, random_state=42)
# sample_fraction_hijab = df_hijab.sample(frac=0.2, random_state=10)
# sample_fraction_student = df_student.sample(frac=0.2, random_state=45)
# sample_fraction_lfw    = df_lfw.sample(frac=0.1, random_state=13)

# # Remove the sampled rows from each DataFrame
# df_38K_remaining   = df_38K.drop(sample_fraction_38K.index)
# df_hijab_remaining = df_hijab.drop(sample_fraction_hijab.index)
# df_student_remaining = df_student.drop(sample_fraction_student.index)
# df_lfw_remaining   = df_lfw.drop(sample_fraction_lfw.index)

# # Merge the remaining rows to form your training set
# df_merged = pd.concat([df_38K_remaining, df_hijab_remaining, df_student_remaining, df_lfw_remaining], ignore_index=True)

# # Merge the sampled rows to form your test set
# df_merged_test = pd.concat([sample_fraction_38K, sample_fraction_hijab, sample_fraction_student, sample_fraction_lfw], ignore_index=True)

df_merged = pd.concat([df_38K, df_hijab], ignore_index=True)
df_merged_test = pd.concat([df_student, df_lfw], ignore_index=True)

In [12]:
df_merged_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54218 entries, 0 to 54217
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image1  54218 non-null  object
 1   image2  54218 non-null  object
 2   label   54218 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [13]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477573 entries, 0 to 477572
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   image1  477573 non-null  object
 1   image2  477573 non-null  object
 2   label   477573 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 10.9+ MB


In [ ]:
# df_merged.to_csv("train_dataset.csv", index=False)
# df_merged_test.to_csv("test_dataset.csv", index=False)

df_merged.to_csv("train_dataset_38K_hijab.csv", index=False)
df_merged_test.to_csv("test_dataset_student_lfw.csv", index=False)

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_df = pd.read_csv("/home/seif_elkerdany/projects/data/train_dataset.csv")
test_df = pd.read_csv("/home/seif_elkerdany/projects/data/test_dataset.csv")

test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42)

test_df.to_csv("/home/seif_elkerdany/projects/data/test_split.csv", index=False)
val_df.to_csv("/home/seif_elkerdany/projects/data/val_split.csv", index=False)

test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27109 entries, 21884 to 15795
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image1  27109 non-null  object
 1   image2  27109 non-null  object
 2   label   27109 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 847.2+ KB


In [5]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27109 entries, 48699 to 30576
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image1  27109 non-null  object
 1   image2  27109 non-null  object
 2   label   27109 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 847.2+ KB
